# 텍스트 분류하기

## 1. 분류기 이용하기

분류를 하고 테스트 진행

### 나이브 베이즈 분류기

베이즈 정리에 기반한 통계적 분류 기법
- 베이즈 정리?
    - p(a|b) = p(a,b) / p(b) = p(b|a)p(a) / p(b)

## 2. 텍스트 분류하기

- 어떤 텍스트가 긍정문일 확률은?
    - p(긍정|입력텍스트) = p(입력텍스트|긍정)p(긍정) / p(입력텍스트)
    - p(부정|입력텍스트) = p(입력텍스트|부정)p(부정) / p(입력텍스트)
----------------------

**나이브** : 개별 단어끼리는 서로 조건부 독립이 성립한다는 순진한 믿음!

**베이즈** : 베이즈 정리

# 영어로 먼저 연습해보기
- 분류기? **NLTK** 이용

- NLTK : 교육용으로 개발된 자연어 처리 및 텍스트 분석을 위한 패키지
    - word_tokenize 와 NaiveBayse 이용


In [4]:
# 필요한 모듈 불러오기

from nltk.tokenize import word_tokenize
import nltk

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\강민정\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# 데이터 입력해주기
# 튜플 형태로 입력

train = [('I like you', 'pos'), ('I hate you', 'neg'), 
         ('I enjoyed it', 'pos'), ('I hate it', 'neg')]
train

[('I like you', 'pos'),
 ('I hate you', 'neg'),
 ('I enjoyed it', 'pos'),
 ('I hate it', 'neg')]

### NLTK 의 Nat-iveBayesClassifier

- p(pos|"I like it") = p("I"|pos)p("like"|pos)p("it"|pos)
- p(neg|"I hate it") = p("I"|neg)p("hate"|neg)p("it"|neg)

## Train 해보기

In [4]:
# 1. train에 있는 문장들의 단어 set 만들기
# set()을 이용하여 집합으로 만들기

all_words = set()  # 빈 집합 선언

for tup in train:
    sent, label = tup[0], tup[1]   # tup[0] : 문장, tup[1] : pos/neg
    words = word_tokenize(sent)
    for word in words:
        all_words.add(word)

all_words

{'I', 'enjoyed', 'hate', 'it', 'like', 'you'}

In [5]:
# 2. 각 문장을 단어가 있는지 없는지 여부 표현
# 단어의 등장 순서 무시 -> 빈도만을 이용!

train_features = []

for tup in train:
    sent, label = tup[0], tup[1]
    words = word_tokenize(sent)
    tmp = dict()
    for set_word in all_words:
        if set_word in words:
            tmp[set_word] = True
        else:
            tmp[set_word] = False
    sent_tup = (tmp, label)
    train_features.append(sent_tup)

# [('I like you', 'pos'), ('I hate you', 'neg'), ('I enjoyed it', 'pos'), ('I hate it', 'neg')]
print(train_features[0])
print(train_features[1])
print(train_features[2])
print(train_features[3])

({'enjoyed': False, 'you': True, 'it': False, 'like': True, 'I': True, 'hate': False}, 'pos')
({'enjoyed': False, 'you': True, 'it': False, 'like': False, 'I': True, 'hate': True}, 'neg')
({'enjoyed': True, 'you': False, 'it': True, 'like': False, 'I': True, 'hate': False}, 'pos')
({'enjoyed': False, 'you': False, 'it': True, 'like': False, 'I': True, 'hate': True}, 'neg')


In [ ]:
# 위의 코드 더 간단하게 나타내기

train_features = []

for tup in train:
    sent, label = tup[0], tup[1]
    words = word_tokenize(sent)
    tmp = dict()
    for set_word in all_words:
        tmp[set_word] = (set_word in words) # set_word가 words에 있는지 여부 판별
    sent_tup = (tmp, label)
    train_features.append(sent_tup)

In [ ]:
# 위의 코드 더더 간단하게 나타내기

train_features = []

for tup in train:
    sent, label = tup[0], tup[1]
    words = word_tokenize(sent)
    tmp = {set_word: (set_word in words) for set_word in all_words} # for문 다음에 오는 명령문을 1줄로 작성
    sent_tup = (tmp, label)
    train_features.append(sent_tup)

In [6]:
# 3) 단어별 확률 계산

classifier = nltk.NaiveBayesClassifier.train(train_features)
classifier.show_most_informative_features()

Most Informative Features
                 enjoyed = False             neg : pos    =      1.7 : 1.0
                    like = False             neg : pos    =      1.7 : 1.0
                       I = True              neg : pos    =      1.0 : 1.0
                      it = False             neg : pos    =      1.0 : 1.0
                      it = True              neg : pos    =      1.0 : 1.0
                     you = False             neg : pos    =      1.0 : 1.0


**결과 해석** : 'enjoyed' 라는 단어가 문장에 없으면 (False), neg:pos = 1.7 : 1.0

+ 확률 구할 때 0.5 씩 더해주기 때문에 비율이 저리 나옵니다!

## Test 해보기

In [7]:
test_sent = 'I like it'

words = word_tokenize(test_sent)
test_feature = {set_word: (set_word in words) for set_word in all_words}

print(test_feature)

{'enjoyed': False, 'you': False, 'it': True, 'like': True, 'I': True, 'hate': False}


- 문장에 사용된 단어 중 한 개의 단어라도 학습 데이터에 없었다면?
    - 전체 확률이 0 이 되어버림! 
    - **>> 라플라스 스무딩을 이용!**
        - 분모, 분자에 일정한 값 (ex 0.5) 를 더해서 분자가 0이 되는 것을 방지

In [8]:
classifier.classify(test_feature)

'pos'

# 한국어 데이터 분류하기

- 영어 버전과 동일하게 작성하면 됨

In [2]:
# 데이터 입력해주기

train = [('사과가 좋아', 'pos'),
        ('밤에 먹는 사과는 비추야', 'neg'),
        ('사과가 잘 익었어 맛있겠다', 'pos')]
train

[('사과가 좋아', 'pos'), ('밤에 먹는 사과는 비추야', 'neg'), ('사과가 잘 익었어 맛있겠다', 'pos')]

In [6]:
# 1. train에 있는 문장들의 단어 set 만들기
# set()을 이용하여 집합으로 만들기

all_words = set()

for tup in train:
    sent, label = tup[0], tup[1]
    words = word_tokenize(sent)
    for word in words:
        all_words.add(word)

print(all_words)

{'먹는', '비추야', '좋아', '잘', '사과는', '익었어', '맛있겠다', '사과가', '밤에'}


In [7]:
# 2. 각 문장을 단어가 있는지 없는지 여부로 표현

train_features = []

for tup in train:
    sent, label = tup[0], tup[1]
    words = word_tokenize(sent)
    tmp = {set_word: (set_word in words) for set_word in all_words}
    sent_tup = (tmp, label)
    train_features.append(sent_tup)

for i in range(len(train_features)):
    print(train_features[i])

({'먹는': False, '비추야': False, '좋아': True, '잘': False, '사과는': False, '익었어': False, '맛있겠다': False, '사과가': True, '밤에': False}, 'pos')
({'먹는': True, '비추야': True, '좋아': False, '잘': False, '사과는': True, '익었어': False, '맛있겠다': False, '사과가': False, '밤에': True}, 'neg')
({'먹는': False, '비추야': False, '좋아': False, '잘': True, '사과는': False, '익었어': True, '맛있겠다': True, '사과가': True, '밤에': False}, 'pos')


In [8]:
# 3. 단어별 확률 계산

classifier = nltk.NaiveBayesClassifier.train(train_features)
classifier.show_most_informative_features()

Most Informative Features
                    맛있겠다 = False             neg : pos    =      1.5 : 1.0
                     익었어 = False             neg : pos    =      1.5 : 1.0
                       잘 = False             neg : pos    =      1.5 : 1.0
                      좋아 = False             neg : pos    =      1.5 : 1.0


## 테스트 하기

In [9]:
test_sent = '사과는 맛있어'

words = word_tokenize(test_sent)
test_feature = {set_word: (set_word in words) for set_word in all_words}

print(test_feature)

{'먹는': False, '비추야': False, '좋아': False, '잘': False, '사과는': True, '익었어': False, '맛있겠다': False, '사과가': False, '밤에': False}


In [10]:
classifier.classify(test_feature)

'neg'

## 학습하기(1) : 형태소 분석기 적용

### 문장을 형태소 단위로 분리해주는 함수 작성


* 처음 문장(raw_sent)
     * '사과가 좋아'
* 형태소 분석 결과 문장(sent)
    * [('사과', 'Noun'), ('가', 'Josa'), ('좋다', 'Adjective')]
* 리턴 문장(' 'join(pos_sent))
    * '사과/Noun 가/Josa 좋다/Adjective'

In [11]:
from konlpy.tag import Okt
okt = Okt()

In [12]:
def pos_tokenize(raw_sent):
    pos_sent = []
    
    # raw_sent: 사과가 좋아
    sent = okt.pos(raw_sent, norm=True, stem=True)    
    # sent: [('사과', 'Noun'), ('가', 'Josa'), ('좋다', 'Adjective')]
    
    for tup in sent:        
        word, tag = tup[0], tup[1]       # tup: ('사과', 'Noun')
        word_tag = word + '/' + tag      # word_tag: '사과/Noun'
        pos_sent.append(word_tag)
    
    return ' '.join(pos_sent)

In [13]:
# train에 있는 문장들의 단어 set 만들기
# set()을 이용하여 집합으로 만들기

all_words = set()

for tup in train:
    sent, label = tup[0], tup[1]    
    sent = pos_tokenize(sent)    # pos_tokenize 함수 추가
    words = word_tokenize(sent)
    for word in words:
        all_words.add(word)

print(all_words)

{'먹다/Verb', '맛있다/Adjective', '에/Josa', '익다/Verb', '사과/Noun', '밤/Noun', '좋다/Adjective', '가/Josa', '자다/Verb', '비추다/Verb', '는/Josa'}


In [14]:
# 학습하기는 같음

train_features = []

for tup in train:
    sent, label = tup[0], tup[1]
    sent = pos_tokenize(sent)    # pos_tokenize 함수 추가
    words = word_tokenize(sent)
    tmp = {set_word: (set_word in words) for set_word in all_words}
    sent_tup = (tmp, label)
    train_features.append(sent_tup)

for i in range(len(train_features)):
    print(train_features[i])

({'먹다/Verb': False, '맛있다/Adjective': False, '에/Josa': False, '익다/Verb': False, '사과/Noun': True, '밤/Noun': False, '좋다/Adjective': True, '가/Josa': True, '자다/Verb': False, '비추다/Verb': False, '는/Josa': False}, 'pos')
({'먹다/Verb': True, '맛있다/Adjective': False, '에/Josa': True, '익다/Verb': False, '사과/Noun': True, '밤/Noun': True, '좋다/Adjective': False, '가/Josa': False, '자다/Verb': False, '비추다/Verb': True, '는/Josa': True}, 'neg')
({'먹다/Verb': False, '맛있다/Adjective': True, '에/Josa': False, '익다/Verb': True, '사과/Noun': True, '밤/Noun': False, '좋다/Adjective': False, '가/Josa': True, '자다/Verb': True, '비추다/Verb': False, '는/Josa': False}, 'pos')


In [15]:
classifier = nltk.NaiveBayesClassifier.train(train_features)
classifier.show_most_informative_features()

Most Informative Features
           맛있다/Adjective = False             neg : pos    =      1.5 : 1.0
                 익다/Verb = False             neg : pos    =      1.5 : 1.0
                 자다/Verb = False             neg : pos    =      1.5 : 1.0
            좋다/Adjective = False             neg : pos    =      1.5 : 1.0


## Test 해보기

In [16]:
test_sent = '사과는 맛있어'

test_sent = pos_tokenize(test_sent)   # pos_tokenize 함수 추가
words = word_tokenize(test_sent)
test_feature = {set_word: (set_word in words) for set_word in all_words}

print(test_feature)

{'먹다/Verb': False, '맛있다/Adjective': True, '에/Josa': False, '익다/Verb': False, '사과/Noun': True, '밤/Noun': False, '좋다/Adjective': False, '가/Josa': False, '자다/Verb': False, '비추다/Verb': False, '는/Josa': True}


In [17]:
classifier.classify(test_feature)

'pos'

# 영화 리뷰 데이터

개발자 도구를 보면 페이지를 넘어갈 때 마다

### 한국어 영화 리뷰 데이터